# Michael Kastanowski, mrk9fx
# Gary Mitchell , gm3gq

## Section 2 Group 7

In [1]:
import csv
import sys
import pandas as pd
import time

from pyspark.sql import SparkSession,SQLContext 
from pyspark.sql.functions import col
from pyspark.sql.functions import when
from pyspark.sql.functions import expr
from pyspark.mllib.linalg import Vectors
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator,RegressionEvaluator, BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline  
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit


import pyspark.sql.functions as F
import pyspark.mllib.regression as reg
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.feature import *  
spark= SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)





In [2]:
vacc_yc= pd.read_csv("Vaccination_Coverage_among_Young_Children__0-35_Months_.csv",sep='\t')

In [3]:
vacc_yc[["Estimate (%)"]]=vacc_yc[["Estimate (%)"]].apply(pd.to_numeric)

In [4]:
vacc_yc #for quick reference

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Age,Estimate (%),95% CI (%),Sample Size
0,DTaP,≥1 Dose,States/Local Areas,Alabama,2011,3 Months,87.6,81.6 to 91.8,218.0
1,DTaP,≥1 Dose,States/Local Areas,Alaska,2011,3 Months,87.9,85.5 to 89.9,288.0
2,DTaP,≥1 Dose,States/Local Areas,Arizona,2011,3 Months,84.8,79.5 to 88.9,266.0
3,DTaP,≥1 Dose,States/Local Areas,Arkansas,2011,3 Months,80.6,75.7 to 84.7,230.0
4,DTaP,≥1 Dose,States/Local Areas,California,2011,3 Months,89.6,84.2 to 93.3,298.0
...,...,...,...,...,...,...,...,...,...
58645,Influenza,NaN,States/Local Areas,Virginia,2017,24 Months,63.2,51.6 to 74.7,201.0
58646,Influenza,NaN,States/Local Areas,Washington,2017,24 Months,72.9,64.3 to 80.8,235.0
58647,Influenza,NaN,States/Local Areas,West Virginia,2017,24 Months,54.0,44.6 to 64.1,175.0
58648,Influenza,NaN,States/Local Areas,Wisconsin,2017,24 Months,68.2,57.9 to 78.1,161.0


In [5]:
df = spark.read.csv("Vaccination_Coverage_among_Young_Children__0-35_Months_.csv",header=True, inferSchema=True, sep = '\t').dropna()

In [6]:
df.show()

+-------+-------+------------------+--------------------+-----------------------+--------+------------+------------+-----------+
|Vaccine|   Dose|    Geography Type|           Geography|Birth Year/Birth Cohort|     Age|Estimate (%)|  95% CI (%)|Sample Size|
+-------+-------+------------------+--------------------+-----------------------+--------+------------+------------+-----------+
|   DTaP|≥1 Dose|States/Local Areas|             Alabama|                   2011|3 Months|        87.6|81.6 to 91.8|        218|
|   DTaP|≥1 Dose|States/Local Areas|              Alaska|                   2011|3 Months|        87.9|85.5 to 89.9|        288|
|   DTaP|≥1 Dose|States/Local Areas|             Arizona|                   2011|3 Months|        84.8|79.5 to 88.9|        266|
|   DTaP|≥1 Dose|States/Local Areas|            Arkansas|                   2011|3 Months|        80.6|75.7 to 84.7|        230|
|   DTaP|≥1 Dose|States/Local Areas|          California|                   2011|3 Months|       

In [7]:
# Estimate to Float
df=df.withColumn("est", col('Estimate (%)').cast("int"))
df=df.withColumn("year", col('Birth Year/Birth Cohort').cast("float"))

In [8]:
df.show()

+-------+-------+------------------+--------------------+-----------------------+--------+------------+------------+-----------+---+------+
|Vaccine|   Dose|    Geography Type|           Geography|Birth Year/Birth Cohort|     Age|Estimate (%)|  95% CI (%)|Sample Size|est|  year|
+-------+-------+------------------+--------------------+-----------------------+--------+------------+------------+-----------+---+------+
|   DTaP|≥1 Dose|States/Local Areas|             Alabama|                   2011|3 Months|        87.6|81.6 to 91.8|        218| 87|2011.0|
|   DTaP|≥1 Dose|States/Local Areas|              Alaska|                   2011|3 Months|        87.9|85.5 to 89.9|        288| 87|2011.0|
|   DTaP|≥1 Dose|States/Local Areas|             Arizona|                   2011|3 Months|        84.8|79.5 to 88.9|        266| 84|2011.0|
|   DTaP|≥1 Dose|States/Local Areas|            Arkansas|                   2011|3 Months|        80.6|75.7 to 84.7|        230| 80|2011.0|
|   DTaP|≥1 Dose|Sta

In [9]:
df = df.withColumn("estBuckets",expr("CASE WHEN est < 10 THEN  0 /*'<10%'*/ \
                                 WHEN est BETWEEN 10 AND 20 THEN 1 /*'10%-20%'*/ \
                                 WHEN est BETWEEN 20 AND 30 THEN 2 /*'20%-30%'*/ \
                                 WHEN est BETWEEN 30 AND 40 THEN 3 /* '30%-40%'*/ \
                                 WHEN est BETWEEN 40 AND 50 THEN 4 /* '40%-50%' */\
                                 WHEN est BETWEEN 50 AND 60 THEN 5 /*'50%-60%'*/ \
                                 WHEN est BETWEEN 60 AND 70 THEN 6 /*'60%-70%'*/ \
                                 WHEN est BETWEEN 70 AND 80 THEN 7 /*'70%-80%'*/ \
                                 WHEN est BETWEEN 80 AND 90 THEN 8 /*'80%-90%'*/ \
                                 ELSE 9 /*'>90%'*/ END AS est_Buckets"))

In [10]:
df.show()

+-------+-------+------------------+--------------------+-----------------------+--------+------------+------------+-----------+---+------+----------+
|Vaccine|   Dose|    Geography Type|           Geography|Birth Year/Birth Cohort|     Age|Estimate (%)|  95% CI (%)|Sample Size|est|  year|estBuckets|
+-------+-------+------------------+--------------------+-----------------------+--------+------------+------------+-----------+---+------+----------+
|   DTaP|≥1 Dose|States/Local Areas|             Alabama|                   2011|3 Months|        87.6|81.6 to 91.8|        218| 87|2011.0|         8|
|   DTaP|≥1 Dose|States/Local Areas|              Alaska|                   2011|3 Months|        87.9|85.5 to 89.9|        288| 87|2011.0|         8|
|   DTaP|≥1 Dose|States/Local Areas|             Arizona|                   2011|3 Months|        84.8|79.5 to 88.9|        266| 84|2011.0|         8|
|   DTaP|≥1 Dose|States/Local Areas|            Arkansas|                   2011|3 Months|    

In [11]:
training, test = df.randomSplit([0.6, 0.4], seed=11)
training.cache()
test.show()

+-------+-------+--------------------+---------+-----------------------+--------+------------+------------+-----------+---+------+----------+
|Vaccine|   Dose|      Geography Type|Geography|Birth Year/Birth Cohort|     Age|Estimate (%)|  95% CI (%)|Sample Size|est|  year|estBuckets|
+-------+-------+--------------------+---------+-----------------------+--------+------------+------------+-----------+---+------+----------+
|   DTaP|≥1 Dose|HHS Regions/National| Region 1|              2014-2015|3 Months|        93.1|91.3 to 94.5|       3073| 93|  null|         9|
|   DTaP|≥1 Dose|HHS Regions/National| Region 1|                   2015|3 Months|        92.6|90.0 to 94.6|       1509| 92|2015.0|         9|
|   DTaP|≥1 Dose|HHS Regions/National| Region 1|              2016-2017|3 Months|        92.3|88.5 to 94.9|       2358| 92|  null|         9|
|   DTaP|≥1 Dose|HHS Regions/National|Region 10|                   2011|3 Months|        87.8|84.2 to 90.7|       1041| 87|2011.0|         8|
|   DT

In [14]:
#Vaccine Dummy Coding

Buckets_string = StringIndexer(inputCol="estBuckets", outputCol="BucketsIndex")
buckets_encoder = OneHotEncoder(inputCol="BucketsIndex", outputCol="BucketVec")

vacc_string = StringIndexer(inputCol="Vaccine", outputCol="VaccineIndex")
vacc_encoder = OneHotEncoder(inputCol="VaccineIndex", outputCol="VaccineVec")

geo_string = StringIndexer(inputCol="Geography", outputCol="GeoIndex")
geo_encoder = OneHotEncoder(inputCol="GeoIndex", outputCol="GeoVec")

age_string=StringIndexer(inputCol="Age", outputCol="AgeIndex")
age_encoder = OneHotEncoder(inputCol="AgeIndex", outputCol="AgeVec")

year_string=StringIndexer(inputCol="Birth Year/Birth Cohort", outputCol="BirthYearIndex")
year_encoder = OneHotEncoder(inputCol="BirthYearIndex", outputCol="BirthYearVec")

assembler =VectorAssembler(inputCols=["Sample Size", 'GeoVec', 'AgeVec', 'VaccineVec', "BirthYearVec", "BucketVec"], outputCol='features')


lr = LinearRegression(featuresCol='features',         # feature vector name
                      labelCol='est',  # target variable name
                      maxIter=10,
                      regParam=0.3, 
                      elasticNetParam=0.8)


paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

#tvs = TrainValidationSplit(estimator=lr,
                           #estimatorParamMaps=paramGrid,
                           #evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           #trainRatio=0.8)



log_reg = LogisticRegression(labelCol='estBuckets', featuresCol='features', maxIter=10, regParam=0.01)


lr_pipeline = Pipeline(stages=[Buckets_string, buckets_encoder, vacc_string, vacc_encoder, geo_string, geo_encoder, age_string, age_encoder, \
                               year_string, year_encoder, assembler, lr] )

log_reg_pipeline = Pipeline(stages=[Buckets_string, buckets_encoder, vacc_string, vacc_encoder, geo_string, geo_encoder, age_string, age_encoder, \
                               year_string, year_encoder, assembler, log_reg] )


lrModel=lr_pipeline.fit(training)
log_Model = log_reg_pipeline.fit(training)

lrPred=lrModel.transform(test)
logPred = log_Model.transform(test)

In [ ]:
ev = RegressionEvaluator(predictionCol="prediction", labelCol="est")
print('-'*20)
print("METRICS")
print("Mean Squared Error:", ev.evaluate(lrPred, {ev.metricName: "mse"}))
print("R Squared:", ev.evaluate(lrPred, {ev.metricName:'r2'}))

In [ ]:
result = logPred.select("estBuckets", "probability", "prediction") \
    .collect()
for row in result:
    print("label= %s -> prob=%s, prediction=%s"
          % (row.estBuckets, row.probability, row.prediction))